In [1]:
###############################################################################
### pthon imports                                                           ###
###############################################################################

from os import path
from requests import get
from bs4 import BeautifulSoup
import os
import pandas as pd
import re

In [2]:
###############################################################################
### local imports                                                           ###
###############################################################################

import zachquire as zaq

In [3]:
def find_file(
    file_name='path/file.txt',
    cache=False,
    cache_age=None
    ):

    if cache==False:
        return None
    
    return path.exists(file_name)

In [4]:
def get_soup(
    url='https://github.com/search?o=desc&p=1&q=advent+of+code&s=stars&type=Repositories',
    headers={'User-Agent': 'Nothing suspicious'},
    file_name='soupfiles/gitsearch.txt',
    cache=False,
    cache_age=None,
    slurper='*'
):
    '''
    
    '''
    # if we already have the data, read it locally
    file_found = find_file(file_name=file_name, cache=cache, cache_age=cache_age)
    if file_found:
        with open(file_name) as f:
            return True, BeautifulSoup(f.read())

    # otherwise go fetch the data
    response = get(url, headers=headers)
    soup = BeautifulSoup(response.text)
    slurps = soup.select(slurper)
    if len(slurps) == 0:
        print('***** NO DATA RETURNED *****')
        return False, soup
    
    # save it for next time
    with open(file_name, 'w') as f:
        f.write(str(slurps[0]))
        if len(slurps)>1:
            for slurp in slurps[1:]:
#                 print(str(slurp))
                f.write('\n' + str(slurp))
        
    with open(file_name) as f:    
        soup = BeautifulSoup(f.read())
#     pd.to_csv(slurps, header=None, index=False)

    return True, soup

In [5]:
slurped, soup = get_soup(
    url='https://github.com/search?o=desc&p=2&q=advent+of+code&s=stars&type=Repositories',
    cache=True,
    slurper='.repo-list-item a',
    file_name='soupfiles/gitsearch.txt'
)

In [6]:
def soup_loop_gitsearch(
    df,
    page_beg=1,
    page_end=20,
    url='https://github.com/',
    adder_prepage='search?o=desc&',
    adder_postpage='&q=advent+of+code&s=stars&type=Repositories',
    headers={'User-Agent': 'Nothing suspicious'},
    directory='soupfiles/',
    file_name='gitsearch',
    file_suffix='.txt',
    cache=False,
    slurper='.repo-list-item a',
):
    for page in range(page_beg, page_end+1):
        use_page = str(page)
        use_url = f'{url}{adder_prepage}p={use_page}{adder_postpage}'
        use_file = f'{directory}{file_name}{use_page:>02s}{file_suffix}'
        print(f'use_page: {use_page:>2s}')
        print(f'use_url: {use_url}')
        print(f'use_file: {use_file}')
        slurped, soup = get_soup(
            url = use_url,
            file_name = use_file,
            cache = cache,
            slurper = slurper,
        )
        log_file = use_file if slurped else None
        loopdict = {
            'soup': [soup], 
            'page': [page], 
            'url': [use_url], 
            'file_name': [log_file]
        }
        df = df.append(pd.DataFrame.from_dict(loopdict), ignore_index=True)
    return df
        

In [7]:
soup_df = pd.DataFrame([], columns=['soup','page','url','file_name'])
# df
soup_df = soup_loop_gitsearch(soup_df, cache=True, page_beg=1, page_end=20)

use_page:  1
use_url: https://github.com/search?o=desc&p=1&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch01.txt
use_page:  2
use_url: https://github.com/search?o=desc&p=2&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch02.txt
use_page:  3
use_url: https://github.com/search?o=desc&p=3&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch03.txt
use_page:  4
use_url: https://github.com/search?o=desc&p=4&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch04.txt
use_page:  5
use_url: https://github.com/search?o=desc&p=5&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch05.txt
use_page:  6
use_url: https://github.com/search?o=desc&p=6&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch06.txt
use_page:  7
use_url: https://github.com/search?o=desc&p=7&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch07.txt
use_page:  8
use_url: https://gith

In [8]:
soup_df

,soup,page,url,file_name
0,"<html><body><a class=""v-align-middle"" data-hyd...",1,https://github.com/search?o=desc&p=1&q=advent+...,soupfiles/gitsearch01.txt
1,"<html><body><a class=""v-align-middle"" data-hyd...",2,https://github.com/search?o=desc&p=2&q=advent+...,soupfiles/gitsearch02.txt
2,"<html><body><a class=""v-align-middle"" data-hyd...",3,https://github.com/search?o=desc&p=3&q=advent+...,soupfiles/gitsearch03.txt
3,"<html><body><a class=""v-align-middle"" data-hyd...",4,https://github.com/search?o=desc&p=4&q=advent+...,soupfiles/gitsearch04.txt
4,"<html><body><a class=""v-align-middle"" data-hyd...",5,https://github.com/search?o=desc&p=5&q=advent+...,soupfiles/gitsearch05.txt
5,"<html><body><a class=""v-align-middle"" data-hyd...",6,https://github.com/search?o=desc&p=6&q=advent+...,soupfiles/gitsearch06.txt
6,"<html><body><a class=""v-align-middle"" data-hyd...",7,https://github.com/search?o=desc&p=7&q=advent+...,soupfiles/gitsearch07.txt
7,"<html><body><a class=""v-align-middle"" data-hyd...",8,https://github.com/search?o=desc&p=8&q=advent+...,soupfiles/gitsearch08.txt
8,"<html><body><a class=""v-align-middle"" data-hyd...",9,https://github.com/search?o=desc&p=9&q=advent+...,soupfiles/gitsearch09.txt
9,"<html><body><a class=""v-align-middle"" data-hyd...",10,https://github.com/search?o=desc&p=10&q=advent...,soupfiles/gitsearch10.txt


In [9]:
print(soup_df.soup[19])

<html><body><a class="v-align-middle" data-hydro-click='{"event_type":"search_result.click","payload":{"page_number":20,"per_page":10,"query":"advent of code","result_position":1,"click_id":47785268,"result":{"id":47785268,"global_relay_id":"MDEwOlJlcG9zaXRvcnk0Nzc4NTI2OA==","model_name":"Repository","url":"https://github.com/kodsnack/advent_of_code_2015"},"client_id":null,"originating_request_id":"C7F4:2C7D:5EBEA74:9ED209C:5DF038B4","originating_url":"https://github.com/search?o=desc&amp;p=20&amp;q=advent+of+code&amp;s=stars&amp;type=Repositories","referrer":null,"user_id":null}}' data-hydro-click-hmac="9da75b1cde3ca6cefec42158aacd558eb5da021c451d01da62dd3d92bfe514b2" href="/kodsnack/advent_of_code_2015">kodsnack/<em>advent</em>_<em>of</em>_<em>code</em>_2015</a>
<a href="http://adventofcode.com" rel="nofollow">http://adventofcode.com</a>
<a class="muted-link" href="/kodsnack/advent_of_code_2015/stargazers">
<svg aria-label="star" class="octicon octicon-star" height="16" role="img" ve

In [10]:
re_url = re.compile(r'"url"\:"(.+?)"')
urllist = re_url.findall(str(soup_df.soup[0]))

urllist

['https://github.com/Bogdanp/awesome-advent-of-code',
 'https://github.com/BurntSushi/advent-of-code',
 'https://github.com/Lysxia/advent-of-coq-2018',
 'https://github.com/fogleman/AdventOfCode2018',
 'https://github.com/mstksg/advent-of-code-2018',
 'https://github.com/Voltara/advent2018-fast',
 'https://github.com/wimglenn/advent-of-code-data',
 'https://github.com/ChrisPenner/Advent-Of-Code-Polyglot',
 'https://github.com/jaksi/advent-of-other-peoples-code',
 'https://github.com/molyswu/hand_detection']

In [11]:
def get_repos_from_gitsearch(df, column='soup', reg_text=r'"url"\:"(.+?)"'):
    git_urls = []
    re_url = re.compile(reg_text)
    for soup in soup_df.soup:
        git_urls.extend(re_url.findall(str(soup)))
    return list(set(git_urls))
        

In [12]:
url_list = get_repos_from_gitsearch(soup_df)

In [13]:
len(url_list)

197

In [14]:
# souprepo = get_soup(
#     url = 'https://github.com/mfikes/advent-of-code',
#     file_name = 'soupfiles/mfikes_advent-of-code.html',
#     slurper='html',
#     cache = False
# )


In [15]:
# souprepo = get_soup(
#     url = 'https://github.com/Diggsey/aoc2018',
#     file_name = 'soupfiles/diggsey_aoc2018.html',
#     slurper='html',
#     cache = False
# )

In [16]:
# souprepo = get_soup(
#     url = 'https://github.com/mhanberg/advent-of-code-elixir-starter',
#     file_name = 'soupfiles/mhanberg_advent-of-code-elixir-starter.html',
#     slurper='html',
#     cache = False
# )

In [17]:
# Library imports for sending request, parsing HTML, and pandas
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
# Example list of URLs from GitHub
urls = [
    "https://github.com/mfikes/advent-of-code",
    "https://github.com/ryanorsinger/intro-to-machine-learning-workshop",
    'https://github.com/Diggsey/aoc2018',
    "https://github.com/kjeliasen/AdventOfCode",
    'https://github.com/mhanberg/advent-of-code-elixir-starter'
]
output = []
for url in urls:
    request = get(url)
    soup = BeautifulSoup(request.text)
    title = url.split("/")[-1]
    author = url.split("/")[-2]
    language_info = soup.select_one(".repository-lang-stats-numbers")
    languages = [language.get_text() for language in language_info.select(".lang")]
    percentages = [percentage.get_text() for percentage in language_info.select(".percent")]
    readme = soup.select_one("#readme article")
    repo_data = {}
    repo_data["author"] = author
    repo_data["title"] = title
    repo_data["readme"] = readme
    repo_data["languages"] = languages
    repo_data["language_percentages"] = percentages
    output.append(repo_data)
df = pd.DataFrame(output)
df.to_csv("soupfiles/all_repo_data.csv")

In [18]:
df

,author,language_percentages,languages,readme,title
0,mfikes,[100.0%],[Clojure],"<article class=""markdown-body entry-content p-...",advent-of-code
1,ryanorsinger,[100.0%],[Jupyter Notebook],"<article class=""markdown-body entry-content p-...",intro-to-machine-learning-workshop
2,Diggsey,[100.0%],[Rust],None,aoc2018
3,kjeliasen,"[97.8%, 2.2%]","[Jupyter Notebook, Python]","<article class=""markdown-body entry-content p-...",AdventOfCode
4,mhanberg,[100.0%],[Elixir],"<article class=""markdown-body entry-content p-...",advent-of-code-elixir-starter


In [19]:
re_repo = re.compile(r'https\://github.com/(.+?)$')
repo_list = []
repo_check = [re_repo.findall(url)[0] for url in url_list]
repo_list.extend(repo_check)
repo_list

['badouralix/advent-of-code-2018',
 'AlexeSimon/adventofcode',
 'def-/nim-advent-of-code-2015',
 'lynn/aoc-2018',
 'molyswu/hand_detection',
 'joelgrus/advent2017',
 'mvaldesdeleon/aoc18',
 'mhanberg/advent-of-code-elixir-starter',
 'mitsuhiko/aoc19',
 'sciyoshi/advent-of-rust-2017',
 'believer/advent-of-code',
 'liuyubobobo/Play-Advent-of-Code',
 'dainnilsson/adventofcode-2017',
 'xSke/aoc15',
 'amochtar/aoc-ranking',
 'KrzaQ/AdventOfCode2018',
 'Bogdanp/awesome-advent-of-code',
 'stephjs/advent-of-code-2019',
 'Ullaakut/aoc19',
 'Sketch-sh/advent-of-code-2018',
 'nicuveo/advent-of-code',
 'dejakobniklas/AdventOfCode',
 'markheath/advent-of-code-js',
 'tginsberg/advent-2017-kotlin',
 'Unix-0x01/AdventOfCode',
 'tginsberg/advent-2018-kotlin',
 'freskog/adventofcode2019',
 'CloudCoders/AdventOfCode2017',
 'benediktwerner/intcode',
 'yamnikov-oleg/adventofcode2017',
 'ploeh/advent-of-code-2017',
 'fogleman/AdventOfCode2018',
 'zero-to-mastery/coding_challenge-24',
 'bhauman/advent-of-clo

In [20]:
filepath = "datafiles/data.json"
zaq.scrape_github_data(repos=repo_list, filepath=filepath)
git_json = pd.read_json(filepath)

badouralix/advent-of-code-2018
https://raw.githubusercontent.com/badouralix/advent-of-code-2018/master/README.md
AlexeSimon/adventofcode
https://raw.githubusercontent.com/AlexeSimon/adventofcode/master/README.md
def-/nim-advent-of-code-2015
None
lynn/aoc-2018
None
molyswu/hand_detection
https://raw.githubusercontent.com/molyswu/hand_detection/temp/README.md
joelgrus/advent2017
https://raw.githubusercontent.com/joelgrus/advent2017/master/README.md
mvaldesdeleon/aoc18
https://raw.githubusercontent.com/mvaldesdeleon/aoc18/master/README.md
mhanberg/advent-of-code-elixir-starter
https://raw.githubusercontent.com/mhanberg/advent-of-code-elixir-starter/master/README.md
mitsuhiko/aoc19
https://raw.githubusercontent.com/mitsuhiko/aoc19/master/README.md
sciyoshi/advent-of-rust-2017
https://raw.githubusercontent.com/sciyoshi/advent-of-rust-2017/master/README.md
believer/advent-of-code
https://raw.githubusercontent.com/believer/advent-of-code/master/README.md
liuyubobobo/Play-Advent-of-Code
https:

albertobastos/advent-of-code-2018-nodejs
https://raw.githubusercontent.com/albertobastos/advent-of-code-2018-nodejs/master/README.md
emmericp/aoc19-intcode-jit
https://raw.githubusercontent.com/emmericp/aoc19-intcode-jit/master/README.md
hsz/AdventOfCode2015
https://raw.githubusercontent.com/hsz/AdventOfCode2015/master/README.md
christofhaerens/advent_of_code
https://raw.githubusercontent.com/christofhaerens/advent_of_code/master/README.md
IdrisTheDragon/AdventOfCode2018
https://raw.githubusercontent.com/IdrisTheDragon/AdventOfCode2018/master/README.md
joelgrus/advent2019
https://raw.githubusercontent.com/joelgrus/advent2019/master/README.md
fhinkel/AdventOfCode2018
https://raw.githubusercontent.com/fhinkel/AdventOfCode2018/master/README.md
mariotacke/advent-of-code-2018
https://raw.githubusercontent.com/mariotacke/advent-of-code-2018/master/README.md
moesol/pony-advent-2016
https://raw.githubusercontent.com/moesol/pony-advent-2016/master/README.md
Larkenx/AdventOfCode2018
https://raw.

emiflake/Advent-of-Code-2019
https://raw.githubusercontent.com/emiflake/Advent-of-Code-2019/master/README.md
roman01la/advent-of-code-2018
https://raw.githubusercontent.com/roman01la/advent-of-code-2018/master/README.md
M0nica/advent-of-code
https://raw.githubusercontent.com/M0nica/advent-of-code/master/README.md
mstksg/advent-of-code-2017
https://raw.githubusercontent.com/mstksg/advent-of-code-2017/master/README.md
natemago/adventofcode2016
None
BurntSushi/advent-of-code
https://raw.githubusercontent.com/BurntSushi/advent-of-code/master/README.md
narimiran/AdventOfCode2017
https://raw.githubusercontent.com/narimiran/AdventOfCode2017/master/README.md
jaksi/advent-of-other-peoples-code
https://raw.githubusercontent.com/jaksi/advent-of-other-peoples-code/master/README.md
bewuethr/advent-of-code
https://raw.githubusercontent.com/bewuethr/advent-of-code/master/README.md
Voltara/advent2018-fast
https://raw.githubusercontent.com/Voltara/advent2018-fast/master/README.md
jcazevedo/advent-of-co

In [21]:
git_json

,author,language,language_all,readme_contents,repo
0,badouralix,Python,"{'Python': 17735580, 'C++': 180994, 'C': 50452...",# Advent of code 2018 solutions\n\n⁣ 🌟 \n ...,badouralix/advent-of-code-2018
1,AlexeSimon,HTML,"{'HTML': 110354, 'Python': 55453}",# adventofcode\n\nThis repo contains my person...,AlexeSimon/adventofcode
2,def-,Nimrod,{'Nim': 10724},None,def-/nim-advent-of-code-2015
3,lynn,Rust,"{'Rust': 10748, 'Idris': 10316, 'Prolog': 8283...",None,lynn/aoc-2018
4,molyswu,Python,{'Python': 34169},using Neural Networks (SSD) on Tensorflow.\n\n...,molyswu/hand_detection
5,joelgrus,Python,{'Python': 73575},# advent2017\n\nLivecoding my way through [Adv...,joelgrus/advent2017
6,mvaldesdeleon,Haskell,{'Haskell': 126402},"# Advent of Code 2018\n\nAdvent of Code 2018, ...",mvaldesdeleon/aoc18
7,mhanberg,Elixir,{'Elixir': 26906},# Advent of Code Elixir Starter\n\nA batteries...,mhanberg/advent-of-code-elixir-starter
8,mitsuhiko,Rust,{'Rust': 22644},# Advent of Code 2019\n\nhttps://adventofcode....,mitsuhiko/aoc19
9,sciyoshi,Rust,"{'Rust': 38289, 'Python': 2936}",# Advent of Rust 2017\n\nThese are my solution...,sciyoshi/advent-of-rust-2017


In [28]:
git_json.repo.value_counts().head()

mstksg/advent-of-code-2018     1
mfikes/advent-of-code          1
dzikoysk/advent-of-code        1
amochtar/aoc-ranking           1
stephjs/advent-of-code-2019    1
Name: repo, dtype: int64

In [23]:
git_json.to_csv('json_test.csv')

In [42]:
# repo = 'kjeliasen/AdventOfCode'
# repo = 'Harleqin/advent-of-code-2017'
repo = 'Bogdanp/awesome-advent-of-code'
reporesp = zaq.github_api_request(f"https://api.github.com/repos/{repo}")


In [43]:
reporesp.json()

{'id': 159918462,
 'node_id': 'MDEwOlJlcG9zaXRvcnkxNTk5MTg0NjI=',
 'name': 'awesome-advent-of-code',
 'full_name': 'Bogdanp/awesome-advent-of-code',
 'private': False,
 'owner': {'login': 'Bogdanp',
  'id': 43347,
  'node_id': 'MDQ6VXNlcjQzMzQ3',
  'avatar_url': 'https://avatars2.githubusercontent.com/u/43347?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/Bogdanp',
  'html_url': 'https://github.com/Bogdanp',
  'followers_url': 'https://api.github.com/users/Bogdanp/followers',
  'following_url': 'https://api.github.com/users/Bogdanp/following{/other_user}',
  'gists_url': 'https://api.github.com/users/Bogdanp/gists{/gist_id}',
  'starred_url': 'https://api.github.com/users/Bogdanp/starred{/owner}{/repo}',
  'subscriptions_url': 'https://api.github.com/users/Bogdanp/subscriptions',
  'organizations_url': 'https://api.github.com/users/Bogdanp/orgs',
  'repos_url': 'https://api.github.com/users/Bogdanp/repos',
  'events_url': 'https://api.github.com/users/Bogdanp/events{

In [38]:
git_json.language.value_counts()

Python         32
Haskell        26
JavaScript     19
Rust           16
Clojure        10
Java            8
Scala           7
Ruby            7
Elixir          7
Kotlin          6
C++             6
Go              6
C#              4
OCaml           4
C               3
R               2
Pony            2
HTML            2
Elm             2
Crystal         1
TSQL            1
Swift           1
Common Lisp     1
Shell           1
Perl            1
Coq             1
PureScript      1
APL             1
F#              1
PHP             1
Nimrod          1
Nim             1
Haxe            1
TypeScript      1
Eiffel          1
Reason          1
Mathematica     1
Racket          1
Name: language, dtype: int64

In [39]:
git_json[git_json.language == 'Common Lisp']

,author,language,language_all,readme_contents,repo
94,Harleqin,Common Lisp,{'Common Lisp': 69390},None,Harleqin/advent-of-code-2017
